# Dataset Overview

let check overview of the dataset.

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session


/kaggle/input/playground-series-s4e8/sample_submission.csv
/kaggle/input/playground-series-s4e8/train.csv
/kaggle/input/playground-series-s4e8/test.csv


In [2]:
train_df = pd.read_csv('/kaggle/input/playground-series-s4e8/train.csv')
test_df = pd.read_csv('/kaggle/input/playground-series-s4e8/test.csv')

In [ ]:
train_df.drop_duplicates(inplace=True)

In [ ]:
# train_df.head().T

In [ ]:
#Target Distribution
# train_df['class'].value_counts()

In [ ]:
#Data Summary
# train_df.describe(include='all').T

# Clean Dataset
Dataset contain a lot of contrived values and so let try clean and see the result.

In [ ]:
#get outlier limit
def get_outlier_range(sr):
    Q3 = sr.quantile(0.75)
    Q1 = sr.quantile(0.25)
    IQR = Q3-Q1
    upper = Q3+(1.5*IQR)
    lower = Q1-(1.5*IQR)
    return lower,upper


#let define some helper function
def clean_data(df):
    #feature columns
    if 'class' in df.columns:
        df = df.drop(columns=['id'])
        feat_cols = df.drop(columns=['class']).columns

#         modify target data to numerical 
        df['class'],_ = df['class'].factorize()
        
    else:
#         df = df.drop(columns=['veil-type'])
        feat_cols = df.drop(columns='id').columns
    
    #category columns
    cat_cols = set([x if y=='object' else np.nan for x,y in zip(feat_cols,df[feat_cols].dtypes)])
    cat_cols.remove(np.nan)
    cat_cols = list(cat_cols)
    cat_cols.sort()
    
    #numerial columns
    num_cols = df[feat_cols].drop(columns=cat_cols).columns
    
    #replace missing values with np.nan
    df = df.fillna(np.nan)
    
    
    #get dictionay of relevant values 
    values ={}
    for col in cat_cols:
#         values[col] = list(df[col].value_counts().apply(lambda x : x if x >100 else np.nan).dropna().index)+['missing']
        values[col] = list(df[col].value_counts().apply(lambda x : x if x >100 else np.nan).dropna().index)+[np.nan]
        

    #modify values with non-relevant values
    for col in cat_cols:
#         df[col] = df[col].fillna('missing')
        df.loc[~df[col].isin(values[col]),col]=np.nan
    
#     #get upper_limit values
#     upper_list = {}
#     for col in num_cols:
#         _,upper_list[col]= get_outlier_range(df[col])

        
        
    #remove rows with outlier values
    for col in num_cols:
        df[col] = pd.qcut(df[col],7,duplicates='drop')
        
#     for col in feat_cols:
#         df[col],_ = pd.factorize(df[col],use_na_sentinel=True)
    

#     one_hot = pd.get_dummies(df[feat_cols],dummy_na=False,dtype=np.uint8)
#     df = pd.concat([df.drop(columns=feat_cols),one_hot],axis=1).astype('category')


    return df


#check data status
def check_data(df):  
    
    #feature columns
    feat_cols = df.drop(columns=['class','id']).columns
    
    #category columns
    cat_cols = set([x if y=='object' else np.nan for x,y in zip(feat_cols,df[feat_cols].dtypes)])
    cat_cols.remove(np.nan)
    cat_cols = list(cat_cols)
    cat_cols.sort()
    
    
    print('### Null Value Counts ###')
    print(df.isnull().sum())
    
    print('### Feature Summary ###')
    for col in feat_cols:
        display(pd.DataFrame(df[col].value_counts()).T)
    
    print('### Numerical Columns Summary ###')
    df[list(num_cols)].describe()


# Pre-process Data

In [ ]:
train  = clean_data(train_df)

In [ ]:
col_list = list(train.columns)

In [ ]:
train.head()

In [ ]:
from sklearn.model_selection import train_test_split
trainX,testX,trainY,testY = train_test_split(train.drop(columns=['class']),train['class'],test_size=0.01,shuffle=True,stratify=train['class'])

# YDF Model

In [ ]:
!pip install ydf -U

In [ ]:
import ydf

In [ ]:
# tuner = ydf.RandomSearchTuner(num_trials=50)
# tuner.choice("sparse_oblique_num_projections_exponent", [0.6,1.,2.,4.])
# tuner.choice("max_depth", [17,25,50])
# tuner.choice("dart_dropout", [0.01,0.05,0.1,0.2,0.4])
# tuner.choice("shrinkage", [0.0001,0.001,0.01,0.1,0.3])

# tuner = ydf.RandomSearchTuner()

# model = ydf.GradientBoostedTreesLearner(label="class").train(pd.concat([trainX,trainY],axis=1))

model = ydf.RandomForestLearner(label="class").train(pd.concat([trainX,trainY],axis=1))

# model = ydf.RandomForestLearner(label="class",
#                                 num_trees=100,winner_take_all=True,
#                                 split_axis='SPARSE_OBLIQUE',sparse_oblique_num_projections_exponent=1.0,
#                                 sparse_oblique_normalization='MIN_MAX',
#                                 honest=True,honest_ratio_leaf_examples=0.6,bootstrap_size_ratio=0.5,
#                                 ).train(pd.concat([trainX,trainY],axis=1))

# Evaluate Model

In [ ]:
y_pred_raw = model.predict(testX)

In [ ]:
from sklearn.metrics import matthews_corrcoef
y_pred = [0 if x<=0.5 else 1 for x in y_pred_raw]
matthews_corrcoef(testY,y_pred)

In [ ]:
from sklearn.metrics import confusion_matrix,ConfusionMatrixDisplay
import seaborn as sns
import matplotlib.pyplot as plt
cf = confusion_matrix(testY, y_pred)
sns.set_context('talk')
disp = ConfusionMatrixDisplay(confusion_matrix=cf)
disp.plot()

plt.show()

# Submission

In [ ]:
#pre-process data
test = clean_data(test_df)

In [ ]:
test.head()

In [ ]:
# pred =[]
# for model in model_list:
#     pred.append([0 if x<=0.5 else 1 for x in model.predict(test.drop(columns=['id'])).squeeze()])

In [ ]:
# sub = pd.DataFrame(['p' if x==1 else 'e' for x in np.median(pred,axis=0) ],index=test['id'],columns=['class'])
# sub.to_csv('submission.csv')

In [ ]:
pred = model.predict(test)

In [ ]:
sub = pd.DataFrame(['p' if x>=0.5 else 'e' for x in pred ],index=test['id'],columns=['class'])
sub.to_csv('submission.csv')

In [ ]:
!head submission.csv